In [30]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import lib_ipmu_daq_config as daq
import lib_ipmu_daq_postprocess as post
import lib_ipmu_daq_graph as graph


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
preset = daq._loadConfig(Path("./_config_preset.toml"))
run = daq._loadConfig(Path("./_config_run.toml"))
config = daq.initParams(preset, run, debug=True)


[Io]
  sample_rate         : 100000
  gen_chunk_sec       : 0.100000
  proc_interval       : 0.125000
  queue_depth         : 40
  quad_depth          : 40

[Gui]
  display_sec         : 3600
  plot_sec            : 0.150000
  gui_interval_ms     : 50
  pruning             : 100

[Logging]
  log_chunk           : 1024
  log_data_num        : 4

[Encoder Postproc]
  quadpulse_width     : 0.000250
  threshold           : 2.500000

[Debug Encoder]
  input_velocity      : 0.100000
  pulse_height        : 5.000000
  pulse_duty          : 0.500000
  pulse_phase_A       : 0.000000

[Debug Power]
  amplitude           : 0.300000
  phase               : 0.000000

[Driver]
  t_DC                : 10
  pps                 : 0
  step                : 100
  rst                 : 1
  target_freq         : 1

[Dependent]
  chunk_sec           : 0.100000
  n_samples_gen       : 10000
  samples_proc        : 12500
  history             : 150
  count_history       : 80
  velo_history        : 80
  pow_

In [ ]:
app = post.DAQApp(config=config, runs_dir=daq.RUNS_DIR)
app.setup() # initialize widgets
gui = graph.DAQGUI(config=config, quad_q=app.quad_q, stop_event=app.stop_event)

# 5. Create the thread objects individually
generator_thread = app.createGeneratorThread()
processor_thread = app.createProcessorThread()

# 6. Start the threads (now controlled from the top level)
print("Starting threads from main...")
generator_thread.start()
processor_thread.start()

gui.run()

generator_thread.join(timeout=0.1)
processor_thread.join(timeout=0.1)



DAQApp instance created. Call setup() to prepare for running.
Setting up DAQApp components...
HDF5 dataset created at: C:\Users\USER\Desktop\ipmu_DAQ\src\..\runs\251003195109.h5
dset shape    = (0, 4)
dset maxshape = (None, 4)
Setup complete.
Starting threads from main...
Processor buffer shape = (1024, 4)
